In [191]:
import json
from datetime import datetime
import numpy as np
import pandas as pd
import ast
import matplotlib.pyplot as plt
import seaborn as sns
import torch
import torch.nn as nn
from torch.utils.data import DataLoader,Dataset
def datacsv2dl(img,cgm,viome,lbl):
    class dataset(Dataset):
        def __init__(self, data, labels):
            self.data = data
            self.labels = labels

        def __len__(self):
            return len(self.data)

        def __getitem__(self, idx):
            d = torch.tensor(self.data.iloc[idx][self.data.columns.difference(['Subject ID','Day'])])
            label = torch.tensor(self.labels.iloc[idx][self.labels.difference(['Subject ID','Day'])], dtype=torch.int32)
            return d, label
    
    img_data = pd.read_csv(img)
    img_data['Image Before Breakfast'] = img_data['Image Before Breakfast'].apply(ast.literal_eval).apply(np.array)
    img_data['Image Before Lunch'] = img_data['Image Before Lunch'].apply(ast.literal_eval).apply(np.array)

    cgm_data = pd.read_csv(cgm)
    cgm_data['Breakfast Time'] = pd.to_datetime(cgm_data['Breakfast Time'], errors='coerce')
    cgm_data['Lunch Time'] = pd.to_datetime(cgm_data['Lunch Time'], errors='coerce')
    cgm_data['CGM Data'] = cgm_data['CGM Data'].apply(ast.literal_eval)
    for _, row in cgm_data.iterrows():
        for i, t in enumerate(row['CGM Data']):
            row['CGM Data'][i] = (datetime.strptime(t[0], '%Y-%m-%d %H:%M:%S').timestamp(), t[1:])

    viome_data = pd.read_csv(viome)
    viome_data['Viome'] = viome_data['Viome'].apply(ast.literal_eval).apply(np.array)

    data = img_data.merge(cgm_data, on=['Subject ID','Day'])
    data = data.merge(viome_data, on=['Subject ID'])

    labels = pd.read_csv(lbl)
    if lbl[-8:]=='only.csv':
        labels = data.merge(labels, on=['Subject ID','Day'])[['Breakfast Calories','Breakfast Carbs','Breakfast Fat','Breakfast Protein']]
    else:
        labels = data.merge(labels, on=['Subject ID','Day'])[['Breakfast Calories','Lunch Calories','Breakfast Carbs','Lunch Carbs','Breakfast Fat','Lunch Fat','Breakfast Protein','Lunch Protein']]
    
    ds = dataset(data,labels)
    return DataLoader(ds, batch_size=32, shuffle=True)

def HM(dt):
    return dt.hour * 60 + dt.minute

def datacsv2ndarry_cgm(cgm,viome,lbl):
    # img_data = pd.read_csv(img)
    # img_data['Image Before Breakfast'] = img_data['Image Before Breakfast'].apply(ast.literal_eval).apply(np.array)
    # img_data['Image Before Lunch'] = img_data['Image Before Lunch'].apply(ast.literal_eval).apply(np.array)

    cgm_data = pd.read_csv(cgm)
    cgm_data['Breakfast Time'] = pd.to_datetime(cgm_data['Breakfast Time'], errors='coerce').apply(HM)
    cgm_data['Lunch Time'] = pd.to_datetime(cgm_data['Lunch Time'], errors='coerce').apply(HM)
    cgm_data['CGM Data'] = cgm_data['CGM Data'].apply(ast.literal_eval)
    cgm_data.dropna(subset=['Breakfast Time','Lunch Time','CGM Data'],inplace=True)
    for j, row in cgm_data.iterrows():
        if cgm_data.at[j, 'CGM Data']==[]:
            cgm_data.drop(j, inplace=True)
            continue
        # print(j)
        # try:
        # cgm_data.at[j, 'Breakfast Time'] = HM(row['Breakfast Time'])
        # cgm_data.at[j, 'Lunch Time'] = HM(row['Lunch Time'])
        for i, t in enumerate(row['CGM Data']):
            row['CGM Data'][i] = [HM(datetime.strptime(t[0], '%Y-%m-%d %H:%M:%S')), t[1]]
        # except:
        #     cgm_data.drop(j, inplace=True)

    viome_data = pd.read_csv(viome)
    # viome_data['Viome'] = viome_data['Viome'].apply(ast.literal_eval).apply(np.array)

    data = cgm_data.merge(viome_data, on=['Subject ID'])

    labels = pd.read_csv(lbl)
    if lbl[-8:]=='only.csv':
        labels = data.merge(labels, on=['Subject ID','Day'])[['Breakfast Calories','Breakfast Carbs','Breakfast Fat','Breakfast Protein']]
        labels = labels[['Breakfast Calories','Breakfast Carbs']]
        print(f'Label Columns: {labels.columns}')
    else:
        labels = data.merge(labels, on=['Subject ID','Day'])[['Breakfast Calories','Lunch Calories','Breakfast Carbs','Lunch Carbs','Breakfast Fat','Lunch Fat','Breakfast Protein','Lunch Protein']]
        labels = labels[['Breakfast Calories','Lunch Calories','Breakfast Carbs','Lunch Carbs']]
        print(f'Label Columns: {labels.columns}')
    data = data[['Breakfast Time','Lunch Time','CGM Data','Diabetes Status','A1C']]
    print(f'Data Columns: {data.columns}')
    
    return np.array(data), np.array(labels)

In [192]:
# train_loader = datacsv2dl('data/img_train.csv','data/cgm_train.csv','data/demo_viome_train.csv','data/label_train.csv')
# test_loader = datacsv2dl('data/img_test.csv','data/cgm_test.csv','data/demo_viome_test.csv','data/label_test_breakfast_only.csv')

print('Training Set:')
X,Y = datacsv2ndarry_cgm('data/cgm_train.csv','data/demo_viome_train.csv','data/label_train.csv')
print('\nTesting Set:')
Xt,Yt = datacsv2ndarry_cgm('data/cgm_test.csv','data/demo_viome_test.csv','data/label_test_breakfast_only.csv')

from sklearn.model_selection import train_test_split
X,Xv,Y,Yv = train_test_split(X,Y,train_size=0.8,random_state=143)

Training Set:
Label Columns: Index(['Breakfast Calories', 'Lunch Calories', 'Breakfast Carbs',
       'Lunch Carbs'],
      dtype='object')
Data Columns: Index(['Breakfast Time', 'Lunch Time', 'CGM Data', 'Diabetes Status', 'A1C'], dtype='object')

Testing Set:
Label Columns: Index(['Breakfast Calories', 'Breakfast Carbs'], dtype='object')
Data Columns: Index(['Breakfast Time', 'Lunch Time', 'CGM Data', 'Diabetes Status', 'A1C'], dtype='object')


In [193]:
X.shape, Xv.shape, Y.shape, Yv.shape

((224, 5), (56, 5), (224, 4), (56, 4))

In [194]:
X

array([[458.0, 722.0,
        list([[460, 138.55333333333334], [465, 153.19333333333333], [470, 172.46666666666667], [475, 193.28666666666666], [480, 217.38], [485, 243.56], [490, 263.73333333333335], [495, 271.0], [500, 270.7266666666667], [505, 267.81333333333333], [510, 258.44666666666666], [515, 246.81333333333333], [520, 239.08666666666667], [525, 231.81333333333333], [530, 223.81333333333333], [535, 216.90666666666667], [540, 214.54666666666665], [545, 217.36666666666667], [550, 222.09333333333333], [555, 225.82], [560, 228.82], [565, 234.00666666666666], [570, 245.28], [575, 256.46], [580, 264.3666666666667], [585, 269.3666666666667], [590, 273.5466666666667], [595, 274.7266666666667], [600, 274.2733333333333], [605, 275.0], [610, 273.9066666666667], [615, 269.6333333333333], [620, 262.72], [625, 249.9], [630, 236.54], [635, 228.63333333333333], [640, 225.27333333333334], [645, 224.36], [650, 217.26666666666668], [655, 206.99333333333334], [660, 197.08666666666667], [665, 190.36

In [195]:
from torch.nn.utils.rnn import pad_sequence, pack_padded_sequence, pad_packed_sequence
# Custom Dataset
class TimeSeriesDataset(Dataset):
    def __init__(self, data, label):
        self.data = data
        self.target = label

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        row = self.data[idx]
        if not row[2]:
            raise ValueError(f"Sequence at index {idx} is empty.")
        seq = torch.tensor(row[2], dtype=torch.float32)
        aux = torch.tensor([row[0], row[1], row[3], row[4]], dtype=torch.float32)
        target = torch.tensor(self.target[idx], dtype=torch.float32)
        return seq, aux, target

# Collate function for DataLoader
def collate_fn(batch):
    sequences, aux_data, targets = zip(*batch)
    lengths = torch.tensor([len(seq) for seq in sequences], dtype=torch.long)
    padded_sequences = pad_sequence(sequences, batch_first=True, padding_value=0.0)
    aux_data = torch.stack(aux_data)
    targets = torch.stack(targets)
    return padded_sequences, lengths, aux_data, targets


dataset = TimeSeriesDataset(X,Y)
dataloader = DataLoader(dataset, batch_size=32, collate_fn=collate_fn)

In [196]:
dataloader.dataset.data.shape, dataloader.dataset.target.shape
# dataloader.dataset.data

((224, 5), (224, 4))

In [197]:
# Model Definition
class RNNWithAuxiliary(nn.Module):
    def __init__(self, input_dim, hidden_dim, aux_dim, output_dim):
        super(RNNWithAuxiliary, self).__init__()
        self.lstm = nn.LSTM(input_dim, hidden_dim, batch_first=True)
        self.aux_dense = nn.Linear(aux_dim, 16)
        self.fc = nn.Linear(hidden_dim + 16, output_dim)

    def forward(self, sequences, lengths, aux_data):
        # Pack sequences for LSTM
        packed_sequences = pack_padded_sequence(sequences, lengths, batch_first=True, enforce_sorted=False)
        packed_output, (hidden, _) = self.lstm(packed_sequences)
        lstm_output, _ = pad_packed_sequence(packed_output, batch_first=True)
        lstm_output = hidden[-1]  # Take the last hidden state
        
        # Process auxiliary data
        aux_output = torch.relu(self.aux_dense(aux_data))
        
        # Concatenate LSTM output and auxiliary data
        combined = torch.cat((lstm_output, aux_output), dim=1)
        output = self.fc(combined)
        return output

In [198]:
# Initialize Model
input_dim = 2  # Each time step has 2 features
hidden_dim = 32
aux_dim = 4  # Auxiliary features
output_dim = 4  # Single output
model = RNNWithAuxiliary(input_dim, hidden_dim, aux_dim, output_dim)

# Define Loss and Optimizer
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

# Training Loop
for epoch in range(10):
    for sequences, lengths, aux_data, targets in dataloader:
        optimizer.zero_grad()
        outputs = model(sequences, lengths, aux_data)
        loss = criterion(outputs.squeeze(), targets)
        loss.backward()
        optimizer.step()
    print(f"Epoch {epoch+1}, Loss: {loss.item():.4f}")

Epoch 1, Loss: 214078.0938
Epoch 2, Loss: 203165.0625
Epoch 3, Loss: 192598.9844
Epoch 4, Loss: 182244.7031
Epoch 5, Loss: 171984.5000
Epoch 6, Loss: 161714.6875
Epoch 7, Loss: 151289.8750
Epoch 8, Loss: 140831.7656
Epoch 9, Loss: 130411.5938
Epoch 10, Loss: 120071.8672
